<a href="https://colab.research.google.com/github/IrenePark123/Test/blob/main/multifactor_yahoo_sim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multi‑Factor (Cholesky) Monte Carlo using Yahoo Finance

This notebook implements a **multi‑factor, Cholesky‑based one‑day Monte Carlo** simulation using **Yahoo Finance** data (via `yfinance`). It is structured into clear sections so you can run top‑to‑bottom or adapt pieces (e.g., add EWMA, 2‑day MPOR, AM/PM split, skew‑t, etc.).

**What you'll get:**
- Data pull and aligned daily log‑returns for **drivers** and **factors**
- Correlation calibration and **extended Cholesky** block construction (Ψ, Φ, Λ)
- Student‑t Monte Carlo for **1‑day horizon**
- Scenario‑level repricing and percentile summaries
- Diagnostics for correlation recovery and λ‑sanity

> Tip: Adjust the ticker lists and horizon in the **Configuration** cell.


## 0) Environment & Dependencies

If you don't have `yfinance` installed in this environment, run the next cell. (It's safe to re‑run.)

In [ ]:
%%capture
%pip install --upgrade yfinance pandas numpy matplotlib

## 1) Imports & Reproducibility

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from numpy.linalg import cholesky, LinAlgError

# Reproducibility
rng = np.random.default_rng(42)

## 2) Configuration (edit me)

- **Drivers**: broad market/sector/macro series
- **Factors**: single names or narrower ETFs you want to simulate
- **Horizon**: one trading day by default
- **Scenarios**: Monte Carlo paths

In [ ]:
# Date range for historical calibration
start = "2016-01-01"
end   = None  # None -> up to today

# Example drivers (adjust to your preference)
driver_tickers = [
    "SPY",         # US large-cap proxy
    "IWM",         # US small-cap proxy
    "EEM",         # EM equities
    "XLK", "XLF",  # Tech, Financials
    "IEF", "HYG",  # 7-10Y UST, High Yield
    "GLD", "SLV",  # Gold, Silver
    "USO", "UNG"   # Oil, NatGas
]

# Example risk factors (single names + thematics)
factor_tickers = [
    "AAPL", "MSFT", "NVDA", "AMZN",
    "TSLA", "META", "JPM", "XOM",
    "QQQ", "TLT"
]

# Simulation settings
nu_driver   = 5      # Student-t df for drivers
nu_factor   = 5      # Student-t df for idiosyncratic components
mpor_days   = 1      # Horizon in trading days (Δt); currently 1-day
n_scenarios = 10000  # Monte Carlo scenario count

## 3) Data Download & Aligned Log‑Returns

Pull **Adj Close** prices, convert to daily **log returns**, and align calendars across drivers & factors. We also filter out columns with too few observations to ensure robust statistics.

In [ ]:
def download_prices(tickers, start=None, end=None):
    df = yf.download(tickers, start=start, end=end, auto_adjust=True, progress=False)["Close"]
    if isinstance(df, pd.Series):
        df = df.to_frame()
    return df.dropna(how="all")

def log_returns(prices: pd.DataFrame) -> pd.DataFrame:
    return np.log(prices).diff().dropna(how="all")

# Download
px_drivers = download_prices(driver_tickers, start, end)
px_factors = download_prices(factor_tickers, start, end)

# Returns and calendar alignment
ret_drivers = log_returns(px_drivers)
ret_factors = log_returns(px_factors)
ret_drivers, ret_factors = ret_drivers.align(ret_factors, join="inner")

# Keep only columns with enough data
min_obs = 252  # ~1 year of trading days
ret_drivers = ret_drivers.dropna(axis=1, thresh=min_obs)
ret_factors = ret_factors.dropna(axis=1, thresh=min_obs)

if ret_drivers.shape[1] == 0 or ret_factors.shape[1] == 0:
    raise ValueError("After filtering for sufficient observations, no driver or factor columns remain. "
                     "Try reducing 'min_obs' or choosing tickers with longer history.")

driver_cols = ret_drivers.columns.tolist()
factor_cols = ret_factors.columns.tolist()

print(f"Drivers ({len(driver_cols)}): {driver_cols}")
print(f"Factors ({len(factor_cols)}): {factor_cols}")

ValueError: After filtering for sufficient observations, no driver or factor columns remain. Try reducing 'min_obs' or choosing tickers with longer history.

## 4) Utilities

- **Nearest correlation (Higham)**: projects a symmetric matrix to the nearest correlation (PSD, unit diagonal).
- **Cross‑correlation**: full Pearson correlation between columns of X (rows) and Y (cols).
- **Vol estimator**: sample daily sigma (swap for EWMA if desired).

In [ ]:
def nearest_correlation_higham(A, tol=1e-12, max_iter=100):
    """Higham (2002) nearest correlation matrix projection (PSD + unit diagonal)."""
    A = (A + A.T) / 2
    n = A.shape[0]
    X = A.copy()
    Y = A.copy()
    deltaS = np.zeros_like(A)

    for _ in range(max_iter):
        R = X - deltaS
        eigvals, eigvecs = np.linalg.eigh(R)
        eigvals_clipped = np.maximum(eigvals, 0.0)
        X = (eigvecs * eigvals_clipped) @ eigvecs.T
        X[np.diag_indices(n)] = 1.0
        deltaS = X - R
        if np.linalg.norm(X - Y, ord="fro") < tol:
            break
        Y = X.copy()
    return X

def cross_corr(X: pd.DataFrame, Y: pd.DataFrame) -> np.ndarray:
    """Full Pearson correlation between columns of X (rows) and Y (cols)."""
    # Standardize
    Xc = (X - X.mean()) / X.std(ddof=1)
    Yc = (Y - Y.mean()) / Y.std(ddof=1)
    n = len(Xc)
    return (Xc.T @ Yc / (n - 1)).T  # shape: (X_cols, Y_cols)

def sample_vols(returns: pd.DataFrame, window=252) -> pd.Series:
    """Daily sigma via sample stdev over a trailing window."""
    daily_sigma = returns.rolling(window).std().dropna().iloc[-1]
    return daily_sigma

## 5) Calibration: Ψ (driver‑driver), Φ (driver‑factor), and Λ (idiosyncratic)

- **Ψ** from Cholesky of the driver‑driver correlation (PSD enforced via Higham if needed)
- **Φ = C<sub>XY</sub> (Ψ<sup>−1</sup>)<sup>T</sup>**
- **λ<sub>j</sub> = √(1 − ‖Φ<sub>j·</sub>‖²)** as the uncorrelated idiosyncratic part per factor

In [ ]:
def calibrate_correlations(RY: pd.DataFrame, RX: pd.DataFrame):
    K = RY.shape[1]
    N = RX.shape[1]

    # Correlations
    C_YY = RY.corr().values              # KxK
    C_XY = cross_corr(RX, RY)            # NxK
    C_XX = RX.corr().values              # NxN (diagnostics only)

    # Ensure PSD for driver-driver; get lower-triangular Psi
    try:
        Psi = cholesky(C_YY).T  # lower-triangular
    except LinAlgError:
        C_YY_psd = nearest_correlation_higham(C_YY)
        Psi = cholesky(C_YY_psd).T

    # Phi = C_XY * (Psi^{-1})^T
    Psi_inv_T = np.linalg.inv(Psi).T
    Phi = C_XY @ Psi_inv_T  # NxK

    # Lambda diagonal from unit-norm constraint
    row_norm_sq = np.sum(Phi**2, axis=1)
    row_norm_sq = np.clip(row_norm_sq, 0.0, 1.0)
    Lambda_diag = np.sqrt(1.0 - row_norm_sq)
    Lambda = np.diag(Lambda_diag)

    blocks = {"C_YY": C_YY, "C_XY": C_XY, "C_XX": C_XX}
    return Psi, Phi, Lambda, blocks

# Daily sigmas (can replace with EWMA if desired)
sigmaY_daily = sample_vols(ret_drivers)
sigmaX_daily = sample_vols(ret_factors)

Psi, Phi, Lambda, corr_blocks = calibrate_correlations(ret_drivers, ret_factors)

print("Psi shape:", Psi.shape)
print("Phi shape:", Phi.shape)
print("Lambda diag (min, max):", float(np.min(np.diag(Lambda))), float(np.max(np.diag(Lambda))))

## 6) Simulation (1‑day)

We simulate one trading day using:
\[ r = \Sigma \cdot L \cdot z \cdot \sqrt{\Delta t} \]
with extended Cholesky
\( L = \begin{bmatrix}\Psi & 0 \\ \Phi & \Lambda \end{bmatrix} \),
daily sigmas on the diagonal of \(\Sigma\), and **Student‑t** noise for drivers and factors.

In [ ]:
def simulate_one_day(Psi, Phi, Lambda,
                     sigmaY: pd.Series,
                     sigmaX: pd.Series,
                     n_scenarios=10000,
                     nu_driver=5, nu_factor=5,
                     annual_trading_days=252):
    K = len(sigmaY)
    N = len(sigmaX)

    # L block
    top = np.hstack([Psi, np.zeros((K, N))])
    bot = np.hstack([Phi, Lambda])
    L = np.vstack([top, bot])  # (K+N) x (K+N)

    # Sigma vector (daily)
    sigma_vec = np.concatenate([sigmaY.values, sigmaX.values])

    # Student-t innovations
    ZY = rng.standard_t(df=nu_driver, size=(n_scenarios, K))
    ZX = rng.standard_t(df=nu_factor, size=(n_scenarios, N))
    Z  = np.concatenate([ZY, ZX], axis=1)

    dt_sqrt = np.sqrt(1.0 / annual_trading_days)

    shocks = Z @ L.T
    scaled = shocks * (sigma_vec * dt_sqrt)  # broadcast

    rY = scaled[:, :K]
    rX = scaled[:, K:]
    return rY, rX

rY_sim, rX_sim = simulate_one_day(
    Psi, Phi, Lambda,
    sigmaY_daily[driver_cols],
    sigmaX_daily[factor_cols],
    n_scenarios=n_scenarios,
    nu_driver=nu_driver,
    nu_factor=nu_factor
)

print("Simulated rY shape:", rY_sim.shape)
print("Simulated rX shape:", rX_sim.shape)

## 7) Re‑price Levels & Summaries

Using last observed prices \(P_t\), one‑day levels are \(P_{t+1}^{(i)} = P_t \cdot e^{r^{(i)}}\). We show percentile summaries for both drivers and factors.

In [ ]:
last_drivers = px_drivers[driver_cols].iloc[-1].values
last_factors = px_factors[factor_cols].iloc[-1].values

pxY_scenarios = last_drivers * np.exp(rY_sim)
pxX_scenarios = last_factors * np.exp(rX_sim)

Y_summary = pd.DataFrame(pxY_scenarios, columns=driver_cols).describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).T
X_summary = pd.DataFrame(pxX_scenarios, columns=factor_cols).describe(percentiles=[0.01,0.05,0.5,0.95,0.99]).T

print("Driver price summary (selected rows):")
display(Y_summary[['mean','std','1%','5%','50%','95%','99%']].round(4))

print("\nFactor price summary (selected rows):")
display(X_summary[['mean','std','1%','5%','50%','95%','99%']].round(4))

## 8) Diagnostics

- **Correlation recovery**: compare simulated against historical targets.
- **λ sanity**: ensure all \(\lambda_j \in [0,1]\).

In [ ]:
# Targets
C_YY_target = corr_blocks["C_YY"]
C_XY_target = corr_blocks["C_XY"]

# Simulated
sim_Y = pd.DataFrame(rY_sim, columns=driver_cols)
sim_X = pd.DataFrame(rX_sim, columns=factor_cols)

C_YY_sim = sim_Y.corr().values
# cross corr: rows=factors, cols=drivers
def _cross_corr_np(X: np.ndarray, Y: np.ndarray) -> np.ndarray:
    Xc = (X - X.mean(axis=0)) / X.std(axis=0, ddof=1)
    Yc = (Y - Y.mean(axis=0)) / Y.std(axis=0, ddof=1)
    return (Xc.T @ Yc / (Xc.shape[0] - 1)).T

C_XY_sim = _cross_corr_np(sim_X.values, sim_Y.values)

mae_driver_driver = float(np.mean(np.abs(C_YY_sim - C_YY_target)))
mae_driver_factor = float(np.mean(np.abs(C_XY_sim - C_XY_target)))

lam_diag = np.diag(Lambda)
lam_min, lam_max = float(lam_diag.min()), float(lam_diag.max())

print(f"MAE corr — driver/driver: {mae_driver_driver:.3f}  |  driver/factor: {mae_driver_factor:.3f}")
print(f"Lambda diag in [min, max] = [{lam_min:.3f}, {lam_max:.3f}]")

## 9) Easy Extensions (placeholders)

- **EWMA volatility**: replace `sample_vols` with an EWMA estimator for more reactive sigmas.
- **2‑day MPOR**: draw a fresh \(z\) for day‑2 and add the day‑1/day‑2 returns.
- **AM/PM split**: simulate with two sub‑intervals per day and aggregate.
- **Skew‑t**: swap the Student‑t sampler with a skewed‑t generator (same block structure).

> If you want a version pre‑wired for EWMA or 2‑day MPOR, tell me your exact tickers and horizon and I’ll generate it.
